In [1]:
import numpy as np
from PIL import Image
import xml.etree.ElementTree as ET
import os
import pandas as pd
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

In [2]:
class DataLoader:
    
    def __init__(self,ann_path,im_path):
        
        self.ann_path = ann_path
        self.im_path = im_path
    
    def get_file_list(self,root, file_type):
        return [os.path.join(directory_path, f) for directory_path, directory_name, 
            files in os.walk(root) for f in files if f.endswith(file_type)]

    def get_train_df(self,ann_path, img_path):
    
        ann_path_list = self.get_file_list(self.ann_path, '.xml')
        ann = np.zeros((len(ann_path_list),4))
        for i in range(len(ann_path_list)):
            a_path = ann_path_list[i]
            root = ET.parse(a_path).getroot()
            ann[i][0] = int(root.find("./object/bndbox/xmin").text)
            ann[i][1] = int(root.find("./object/bndbox/ymin").text)
            ann[i][2] = int(root.find("./object/bndbox/xmax").text)
            ann[i][3] = int(root.find("./object/bndbox/ymax").text)
        return ann

    def get_image_data(self):
    
        image_list = get_file_list(self.im_path,'png')
        image_data = [ cv2.imread(image_path) for image_path in image_list]

        return image_data
    
    def resize_image_bounding_box(self):
        
        image_data = self.get_image_data()
        targetSize = (100,100)
        image_list = get_file_list(self.im_path,'png')
        resized_image_list = []
        for i in range(len(image_data)):
            x_scale = 100/image_data[i].shape[0]
            y_scale = 100/image_data[i].shape[1]
            train_box[i][0] = int(np.round(train_box[i][0]*x_scale))
            train_box[i][1] = int(np.round(train_box[i][1]*y_scale))
            train_box[i][2] = int(np.round(train_box[i][2]*x_scale))
            train_box[i][3] = int(np.round(train_box[i][3]*y_scale))
            image = cv2.imread(image_list[i])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) ## grayscale image
            norm_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) ## normalize
            image = cv2.resize(norm_image,targetSize)  ## resize to 100*100
            #print(image.shape)
            image = np.array(image)
            resized_image_list.append(image.ravel())

        return train_box,np.array(resized_image_list)  
    
    def train_test_split(self):
        
        train_box,resized_image_list = self.resize_image_bounding_box()
        return train_test_split(resized_image_list, train_box, test_size=0.3, random_state=34)
    

In [3]:
import math

class LinearRegression():
    
    def __init__(self, learning_rate,alpha,regularization,L1_ratio):
            
        self.param = None
        self.learning_rate = learning_rate
        self.reg_rate = alpha
        self.regularization = regularization
        self.L1_ratio = L1_ratio
        self.n_iterations = 2000
    def initialize_parameters(self, input_dim,output_dim):
        
        self.param = np.ones((input_dim,output_dim))
        
    def fit(self, x, y):
        
        l1_ratio = self.L1_ratio
        x = np.insert(x, 0, 1, axis=1)
        self.training_errors = []
        self.MAE = []
        self.initialize_parameters(input_dim = x.shape[1],output_dim = y.shape[1])
        print(self.param.shape)
        # Do gradient descent for n_iterations
        for i in tqdm(range(self.n_iterations)):
            
            self.learning_rate = self.learning_rate/(10**int((i/1000))) ## learning rate decay
            y_pred = np.dot(x,self.param)
            if self.regularization == None:
                mse = np.mean(0.5 * (y - y_pred)**2)
                grad_param = np.dot(x.T,(y_pred - y))
            elif self.regularization == 'L1':
                mse = np.mean(0.5 * (y - y_pred)**2) + self.reg_rate*np.linalg.norm(self.param, ord= 1)
                grad_param = np.dot(x.T,(y_pred - y)) + self.reg_rate*np.sign(self.param)
            elif self.regularization == 'L2':
                mse = np.mean(0.5 * (y - y_pred)**2) + 0.5*self.reg_rate*np.linalg.norm(self.param)**2
                grad_param = np.dot(x.T,(y_pred - y)) + self.reg_rate*self.param
            elif self.regularization == 'Elastic':
                mse = np.mean(0.5 * (y - y_pred)**2) + self.reg_rate*(l1_ratio*np.linalg.norm(self.param, ord= 1)+(1-l1_ratio)*0.5*np.linalg.norm(self.param)**2)
                grad_param = np.dot(x.T,(y_pred - y)) + self.reg_rate*((1-l1_ratio)*self.param + l1_ratio*np.sign(self.param))
            
            self.training_errors.append(mse)
            self.MAE.append(np.mean(np.abs(y-y_pred)))
            self.param = self.param - self.learning_rate * grad_param
        
            
    def predict(self,x_test):
        x_test = np.insert(x_test, 0, 1, axis=1)
        pred = np.dot(x_test,self.param)
        return pred
    
    def getMetrics(self,y_pred,actual):
        
        MoU = 0
        for i in range(y_pred.shape[0]):
            if y_pred[i][2] < actual[i][0] or y_pred[i][0] > actual[i][2] or y_pred[i][3] < actual[i][1] or y_pred[i][1] > actual[i][3]:
                continue
            x_min = max(y_pred[i][0],actual[i][0])
            x_max = min(y_pred[i][2],actual[i][2])
            y_min = max(y_pred[i][1],actual[i][1])
            y_max = min(y_pred[i][3],actual[i][3])
            intersection = (x_max - x_min)*(y_max-y_min)
            union = (y_pred[i][2]-y_pred[i][0])*(y_pred[i][3]-y_pred[i][1]) + (actual[i][2]-actual[i][0])*(actual[i][3]-actual[i][1]) - intersection;
            MoU += intersection/union
            
        MSE = np.mean(0.5 * (actual - y_pred)**2)
        MAE = np.mean(np.abs(actual - y_pred))
        
        return MSE,MoU/y_pred.shape[0],MAE
    
    def plot_loss(self):
    
        iters = np.arange(0,self.n_iterations,1)
        fig = plt.figure(figsize=(4,4))
        plt.plot(iters[50:],self.training_errors[50:],label = 'MSE')
        plt.plot(iters[50:],self.MAE[50:],label = 'MAE')
        plt.xlabel('No. of iterations')
        plt.ylabel('Loss')
        plt.legend()
        if self.regularization == None:
            plt.title('Simple Linear Regression')
            plt.savefig('Q3_LinearRegression.png')
        else:
            plt.title(f'{self.regularization}Regression')
            plt.savefig(f'Q3{self.regularization}.png')
        plt.close()

In [4]:
if __name__ == '__main__':
    
    '''
    ann_path = '../Assignment 1/data/Road Sign Detection/annotations/'
    image_path = '../Assignment 1/data/Road Sign Detection/images/'
    dataset = DataLoader(ann_path,image_path)
    X_train, X_test, y_train, y_test = dataset.train_test_split()
    data = [X_train, X_test, y_train, y_test]
    
    with open('Q3.pkl', 'wb') as outfile:
        pickle.dump(data, outfile, pickle.HIGHEST_PROTOCOL)
    '''
        
    with open('Q3.pkl', 'rb') as infile:
        result = pickle.load(infile)
    
    X_train, X_test, y_train, y_test = result
    
    
    clf = LinearRegression(1e-7,None,None,None)
    clf.fit(X_train,y_train)
    training_errors = np.array(clf.training_errors)
    training_MAE = np.array(clf.MAE)
    y_pred = clf.predict(X_test)
    MSE,mIoU,MAE = clf.getMetrics(y_pred,y_test) 
    print(f'metrics for Simple Linear are {MSE,mIoU,MAE}')
    clf.plot_loss()
    
    
    clf = LinearRegression(1e-7,0.1,'L1',None)
    clf.fit(X_train,y_train)
    training_errors = np.array(clf.training_errors)
    training_MAE = np.array(clf.MAE)
    y_pred = clf.predict(X_test)
    MSE,mIoU,MAE = clf.getMetrics(y_pred,y_test) 
    print(f'metrics for L1 are {MSE,mIoU,MAE}')
    clf.plot_loss()
    
    clf = LinearRegression(1e-7,0.1,'L2',None)
    clf.fit(X_train,y_train)
    training_errors = np.array(clf.training_errors)
    training_MAE = np.array(clf.MAE)
    y_pred = clf.predict(X_test)
    MSE,mIoU,MAE = clf.getMetrics(y_pred,y_test) 
    print(f'metrics for L2 are {MSE,mIoU,MAE}')
    clf.plot_loss()
    
    clf = LinearRegression(1e-7,0.1,'Elastic',0.1)
    clf.fit(X_train,y_train)
    training_errors = np.array(clf.training_errors)
    training_MAE = np.array(clf.MAE)
    y_pred = clf.predict(X_test)
    MSE,mIoU,MAE = clf.getMetrics(y_pred,y_test) 
    print(f'metrics for Elastic are {MSE,mIoU,MAE}')
    clf.plot_loss()
    
    ## L1 ratio tuning for Elastic Net
    L1_ratio = 0.1
    while(L1_ratio<1):
        clf = LinearRegression(1e-7,0.1,'Elastic',L1_ratio)
        clf.fit(X_train,y_train)
        training_errors = np.array(clf.training_errors)
        training_MAE = np.array(clf.MAE)
        y_pred = clf.predict(X_test)
        MSE,mIoU,MAE = clf.getMetrics(y_pred,y_test) 
        print(f'metrics for Elastic with L1 ratio = {L1_ratio} are {MSE,mIoU,MAE}')
        L1_ratio += 0.2
    
    ## L1 regression Regularization rate tuning
    reg_rate = 0.001
    while(reg_rate<1):
        clf = LinearRegression(1e-7,reg_rate,'L1',None)
        clf.fit(X_train,y_train)
        training_errors = np.array(clf.training_errors)
        training_MAE = np.array(clf.MAE)
        y_pred = clf.predict(X_test)
        MSE,mIoU,MAE = clf.getMetrics(y_pred,y_test) 
        print(f'metrics for L1 with lambda = {reg_rate} are {MSE,mIoU,MAE}')
        reg_rate *= 10
    
    
    ##  L1 regression Regularization rate tuning
    
    reg_rate = 0.001
    while(reg_rate<1):
        clf = LinearRegression(1e-7,reg_rate,'L2',None)
        clf.fit(X_train,y_train)
        training_errors = np.array(clf.training_errors)
        training_MAE = np.array(clf.MAE)
        y_pred = clf.predict(X_test)
        MSE,mIoU,MAE = clf.getMetrics(y_pred,y_test) 
        print(f'metrics for L2 with lambda = {reg_rate} are {MSE,mIoU,MAE}')
        reg_rate *= 10
    
    ## Elastic Net hyper parameter tuning
    
    reg_rate = 0.001
    while(reg_rate<1):
        clf = LinearRegression(1e-7,reg_rate,'Elastic',0.1)
        clf.fit(X_train,y_train)
        training_errors = np.array(clf.training_errors)
        training_MAE = np.array(clf.MAE)
        y_pred = clf.predict(X_test)
        MSE,mIoU,MAE = clf.getMetrics(y_pred,y_test) 
        print(f'metrics for Elastic with lambda = {reg_rate} are {MSE,mIoU,MAE}')
        reg_rate *= 10
    

(10001, 4)


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [02:26<00:00, 13.61it/s]


metrics for Simple Linear are (20.378668677854282, 0.0, 4.745390287690216)
(10001, 4)


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [02:10<00:00, 15.29it/s]


metrics for L1 are (20.368501106409237, 0.0, 4.744235568115445)
(10001, 4)


...

metrics for L1 with lambda = 0.001 are (20.378566983748193, 0.0, 4.745378739940554)
(10001, 4)


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [02:21<00:00, 14.09it/s]


metrics for L1 with lambda = 0.01 are (20.377651753231454, 0.0, 4.745274810651416)
(10001, 4)


 58%|████████████████████████████████████████████▉                                 | 1151/2000 [01:22<01:00, 13.94it/s]


KeyboardInterrupt: 

In [ ]:
annotation_path = 
image_path = 

In [ ]:
train_box = get_train_df(annotation_path,image_path)
print(train_box)

In [ ]:
image_data = get_image_data(image_path)
print(image_data[0].shape)

In [ ]:
train_box,resized_image_list = resize_image_bounding_box(image_data)
print(resized_image_list.shape)
print(train_box.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(resized_image_list, train_box, test_size=0.3, random_state=34)
print(X_train.shape,y_train.shape)